In [ ]:
// =======================================
// LANDSAT 8 - LST Lembang Analysis
// =======================================

// 1. Panggil Image Collection Landsat 8 Surface Reflectance Tier 1
var Landsat_8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2");

// 2. Filter berdasarkan lokasi dan rentang waktu
var L8_LEMBANG = Landsat_8
  .filterBounds(lembang2)
  .filterDate('2012-01-01', '2022-12-31');

// 3. Cloud Masking Function
var masking = function(img) {
  var cloudShadowBitMask = (1 << 4); // Bit 4
  var cloudBitMask = (1 << 3);       // Bit 3
  var qa = img.select('QA_PIXEL');
  var maskShadow = qa.bitwiseAnd(cloudShadowBitMask).eq(0);
  var maskCloud = qa.bitwiseAnd(cloudBitMask).eq(0);
  var mask = maskShadow.and(maskCloud);
  return img.updateMask(mask).copyProperties(img, ['system:time_start']);
};

// 4. Terapkan Masking & Median Composite
var L8clear = L8_LEMBANG
  .sort('CLOUD_COVER_LAND')
  .map(masking)
  .median();

// 5. Klip citra hasil masking dengan wilayah lembang2
var LEMBANG_CLIP = L8clear.clip(lembang2);

// 6. Tampilkan Citra Dasar
Map.addLayer(LEMBANG_CLIP);

// 7. Tampilkan Citra Komposit (dengan visualisasi jika sudah didefinisikan)
Map.addLayer(LEMBANG_CLIP, imageVisParam, 'True Colour');

// 8. Scaling Factor untuk Band Reflektif dan Termal
var applyScaleFactors = function(image) {
  var opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
  var thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);
  return image
    .addBands(opticalBands, null, true)
    .addBands(thermalBands, null, true);
};

var LembangScale = applyScaleFactors(LEMBANG_CLIP);

// 9. Ambil Band Termal dan Konversi ke Celsius
var Thermal_Band = LembangScale.select('ST_B10');
var LST_celcius = Thermal_Band.subtract(273);

// 10. Tampilkan Layer LST
var LSTParams = {
  min: 27,
  max: 40,
  palette: ['blue', 'limegreen', 'yellow', 'orange', 'red']
};
Map.addLayer(LST_celcius, LSTParams, 'LST image');

// 11. Klasifikasi Nilai LST
var LLA = LST_celcius.lte(27).selfMask();
var LA = LST_celcius.gt(27).and(LST_celcius.lte(30.25)).selfMask();
var NA = LST_celcius.gt(30.25).and(LST_celcius.lte(33.5)).selfMask();
var HA = LST_celcius.gt(33.5).and(LST_celcius.lte(36.75)).selfMask();
var HHA = LST_celcius.gt(36.75).selfMask();

// 12. Tampilkan Layer Klasifikasi LST
Map.addLayer(LLA, imageVisParam2, 'Very Low LST Areas');
Map.addLayer(LA, imageVisParam3, 'Low LST Areas');
Map.addLayer(NA, imageVisParam4, 'Normal LST Areas');
Map.addLayer(HA, imageVisParam5, 'High LST Areas');
Map.addLayer(HHA, imageVisParam6, 'Very High LST Areas');

// =======================================
// KOMENTAR: Bagian Export & Chart (Dinonaktifkan)
// =======================================

/*
Export.image.toDrive({
  image: LLA,
  description: 'LLA_LST',
  region: lembang2,
  scale: 30
});

Export.image.toDrive({
  image: LA,
  description: 'LA_LST',
  region: lembang2,
  scale: 30
});

Export.image.toDrive({
  image: NA,
  description: 'NA_LST',
  region: lembang2,
  scale: 30
});

Export.image.toDrive({
  image: HA,
  description: 'HA_LST',
  region: lembang2,
  scale: 30
});

Export.image.toDrive({
  image: HHA,
  description: 'HHA_LST',
  region: lembang2,
  scale: 30
});

Export.image.toDrive({
  image: LST_celcius,
  description: 'LST_full',
  region: lembang2,
  scale: 30
});

var ts1 = ui.Chart.image.series({
  imageCollection: Thermal_Band,
  region: lembang2,
  reducer: ee.Reducer.mean(),
  scale: 1000,
  xProperty: 'system:time_start'
}).setOptions({
  title: 'Band 5 & 4 Landsat 8',
  vAxis: { title: 'LST Celsius' }
});
print(ts1);
*/

// =======================================
// END
// =======================================
